<a href="https://colab.research.google.com/github/jeffreyfeng99/SYDE_522_A3/blob/master/SYDE_522_Assignment_3_testingincluded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Submission Notes
Group members:

Jeffrey Feng, 20704800

Joey Kuang, 20726074

# Import statements

In [10]:
import os
import pandas as pd
import numpy as np
import random
from PIL import Image
from tqdm import tqdm
from datetime import datetime

import torch
import torch.nn as nn
import torch.utils.data as data
from torch.autograd import Function
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torchvision import datasets
from torchvision import models
import torch.optim as optim

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [4]:
cuda = True
cudnn.benchmark = True
LR = 1e-3
BATCH_SIZE = 64
IMAGE_SIZE = 224 #227
FT_OUT_SIZE = 512
N_EPOCH = 10 #100

# Data loader

In [5]:
dataset_root = '/content/drive/MyDrive/4B/SYDE-522/data'
source_dataset_name = 'train_set'
target_dataset_name = 'test_set'

source_image_root = os.path.join(dataset_root, source_dataset_name)
target_image_root = os.path.join(dataset_root, target_dataset_name)

train_label_list = os.path.join(dataset_root, 'train_labels.csv')

In [6]:
class GetLoader(data.Dataset):
    def __init__(self, data_root, data_list=None, transform=None):
        self.root = data_root
        self.transform = transform

        # we only pass data_list if it's training set
        if data_list is not None:
            df = pd.read_csv(data_list)
            self.img_paths = df['dir'].to_list()

            if 'label2' in df.columns:
                self.img_labels = df['label2'].to_list()
            else: 
                self.img_labels = ['0' for i in range(len(self.img_paths))]
        else:
            # Walk through test folder - we don't need labels
            self.img_paths = [f for root,dirs,files in os.walk(data_root) for f in files if f.endswith('.png')]
            self.img_labels = ['0' for i in range(len(self.img_paths))]

        self.n_data = len(self.img_paths)

    def __getitem__(self, item):
        img_paths, labels = self.img_paths[item%self.n_data], self.img_labels[item%self.n_data]
        imgs = Image.open(os.path.join(self.root, img_paths)).convert('RGB')

        if self.transform is not None:
            imgs = self.transform(imgs)
            labels = int(labels)

        return imgs, labels, img_paths

    def __len__(self):
        return self.n_data

In [7]:
# Preprocess data
def preprocess_fn(mu=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)):
  img_transform = transforms.Compose([
      transforms.Resize(IMAGE_SIZE),
      transforms.ToTensor(),
      transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) # why isn't this (0.1307,) and (0.3081,) like in test?
  ])

  return img_transform

def prep_dataloader(image_root, label_list=None, img_transform=None, 
                    drop_last=False, shuffle=True):
    dataset = GetLoader(
        data_root=image_root,
        data_list=label_list,
        transform=img_transform
    )

    dataloader = data.DataLoader(
        dataset=dataset,
        batch_size=BATCH_SIZE,
        shuffle=shuffle,
        num_workers=8,
        drop_last=drop_last)
    
    return dataset, dataloader


# Model definition

In [8]:
# if False, then we are feature extracting
def set_parameter_requires_grad(model, finetune):
    for param in model.parameters():
        param.requires_grad = finetune

In [9]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None


class CNNModel(nn.Module):

    def __init__(self, model_name="resnet18"):
        super(CNNModel, self).__init__()

        if model_name == "resnet18":
            self.feature = models.resnet18(pretrained=True) 
            self.feature.fc = nn.Identity()
        elif model_name == "vgg11":
            self.feature = models.vgg11_bn(pretrained=True) 
            self.feature.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1)) # original is (7,7)
            self.feature.classifier = nn.Identity()
        else:
            # Need some default model?
            self.feature = nn.Sequential()
            self.feature.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
            self.feature.add_module('f_bn1', nn.BatchNorm2d(64))
            self.feature.add_module('f_pool1', nn.MaxPool2d(2))
            self.feature.add_module('f_relu1', nn.ReLU(True))
            self.feature.add_module('f_conv2', nn.Conv2d(64, 128, kernel_size=3))
            self.feature.add_module('f_bn2', nn.BatchNorm2d(128))
            self.feature.add_module('f_drop1', nn.Dropout2d())
            self.feature.add_module('f_pool2', nn.MaxPool2d(2))
            self.feature.add_module('f_relu2', nn.ReLU(True))
            self.feature.add_module('f_conv3', nn.Conv2d(128, 256, kernel_size=3))
            self.feature.add_module('f_bn3', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop3', nn.Dropout2d())
            self.feature.add_module('f_pool3', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv4', nn.Conv2d(256, 256, kernel_size=3))
            self.feature.add_module('f_bn4', nn.BatchNorm2d(256))
            self.feature.add_module('f_drop4', nn.Dropout2d())
            self.feature.add_module('f_pool4', nn.MaxPool2d(2))
            self.feature.add_module('f_relu4', nn.ReLU(True))
            self.feature.add_module('f_conv5', nn.Conv2d(256, 512, kernel_size=3))
            self.feature.add_module('f_bn5', nn.BatchNorm2d(512))
            self.feature.add_module('f_drop5', nn.Dropout2d())
            self.feature.add_module('f_pool5', nn.MaxPool2d(2))
            self.feature.add_module('f_relu5', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout2d())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 7))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax(dim=1))

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(FT_OUT_SIZE, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 4))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax(dim=1))

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, IMAGE_SIZE, IMAGE_SIZE)
        feature = self.feature(input_data)
        feature = feature.view(-1, FT_OUT_SIZE)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

# Test pipeline

In [18]:
def test(net, epoch):

    # load data
    img_transform_source = preprocess_fn(mu=(0.1307,), std=(0.3081,))
    img_transform_target = preprocess_fn()

    dataset_source, dataloader_source = prep_dataloader(
        image_root=os.path.join(source_image_root, 'train_set'),
        label_list=train_label_list,
        img_transform=img_transform_source,
        shuffle=False
    )

    dataset_target, dataloader_target = prep_dataloader(
        image_root=os.path.join(target_image_root, 'test_set'),
        img_transform=img_transform_target,
        shuffle=False
    )

    net.eval()

    if cuda:
        net = net.cuda()

    train_pths, train_preds = inference(net, dataloader_source, cuda=cuda, alpha=alpha)
    train_results = pd.DataFrame({'id': train_pths, 'label': train_preds})
    os.makedirs('prediction_results', exist_ok=True)
    train_results.to_csv('prediction_results/%s_train_epoch%s.csv' % (datetime.now().strftime("%m%d%Y"), epoch), index=False)

    test_pths, test_preds = inference(net, dataloader_target, cuda=cuda, alpha=alpha)
    test_results = pd.DataFrame({'id': test_pths, 'label': test_preds})
    test_results.to_csv('prediction_results/%s_test_epoch%s.csv' % (datetime.now().strftime("%m%d%Y"), epoch), index=False)

    print('epoch: %d, accuracy of the train dataset: %f' % (epoch, compare(train_label_list, 'prediction_results/%s_train_epoch%s.csv' % (datetime.now() .strftime("%m%d%Y"), epoch))))

    # Secret
    test_label_list = os.path.join(dataset_root, 'dummy_test_labels.csv')
    print('epoch: %d, accuracy of the test dataset: %f' % (epoch, compare(test_label_list, 'prediction_results/%s_test_epoch%s.csv' % (datetime.now() .strftime("%m%d%Y"), epoch))))

def inference(net, dataloader, cuda=True, alpha=0.0):
    preds = []
    pths = []
    for input_img, _, img_paths in dataloader: 

        if cuda:
            input_img = input_img.cuda()

        class_output, _ = net(input_data=input_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        pths = pths + list(img_paths)
        preds = preds + list(pred.squeeze(1).cpu().numpy())
    return pths, preds

def compare(true_labels, predicted_labels):
    combined_df = pd.read_csv(true_labels)
    predicted_df = pd.read_csv(predicted_labels)

    combined_df['label'] = combined_df['dir'].map(predicted_df.set_index('id')['label'])

    true_labels = np.array(combined_df['label2'].to_list())
    pred_labels = np.array(combined_df['label'].to_list())

    return np.sum(true_labels == pred_labels) / len(true_labels)

# Training pipeline

In [19]:
manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data
img_transform_source = preprocess_fn()
img_transform_target = preprocess_fn()

dataset_source, dataloader_source = prep_dataloader(
    image_root=os.path.join(source_image_root, 'train_set'), # TODO should we unnest
    label_list=train_label_list,
    img_transform=img_transform_source
)

dataset_target, dataloader_target = prep_dataloader(
    image_root=os.path.join(target_image_root, 'test_set'),
    img_transform=img_transform_target
)

# load model
my_net = CNNModel()

# setup optimizer
optimizer = optim.Adam(my_net.parameters(), lr=LR)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

set_parameter_requires_grad(my_net, True)

# training
for epoch in tqdm(range(N_EPOCH)):

    len_dataloader = max(len(dataloader_source), len(dataloader_target))
    data_source_iter = iter(dataloader_source)
    data_target_iter = iter(dataloader_target)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / N_EPOCH / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = data_source_iter.next()
        s_img, s_label, _ = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)

        class_output, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_s_label = loss_class(class_output, class_label)
        err_s_domain = loss_domain(domain_output, domain_label)

        # training model using target data
        if i == len(dataloader_target):
            data_target_iter = iter(dataloader_target)
        data_target = data_target_iter.next()
        t_img, _, _ = data_target

        batch_size = len(t_img) # TODO: why?

        input_img = torch.FloatTensor(batch_size, 3, IMAGE_SIZE, IMAGE_SIZE)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)

        _, domain_output = my_net(input_data=input_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domain_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1

        print('epoch: %d, [iter: %d / all %d], err_s_label: %f, err_s_domain: %f, err_t_domain: %f' \
            % (epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    # torch.save(my_net, 'output/model_epoch_{}.pth'.format(epoch))
    test(my_net, epoch)
    my_net.train()

print('done')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0%|          | 0/10 [00:00<?, ?it/s]

epoch: 0, [iter: 1 / all 95], err_s_label: 2.008929, err_s_domain: 1.542843, err_t_domain: 1.478157
epoch: 0, [iter: 2 / all 95], err_s_label: 1.796367, err_s_domain: 1.486581, err_t_domain: 1.427461
epoch: 0, [iter: 3 / all 95], err_s_label: 1.565000, err_s_domain: 1.430585, err_t_domain: 1.356369
epoch: 0, [iter: 4 / all 95], err_s_label: 1.558840, err_s_domain: 1.395514, err_t_domain: 1.325767
epoch: 0, [iter: 5 / all 95], err_s_label: 1.434266, err_s_domain: 1.345569, err_t_domain: 1.297536
epoch: 0, [iter: 6 / all 95], err_s_label: 1.484700, err_s_domain: 1.323253, err_t_domain: 1.251182
epoch: 0, [iter: 7 / all 95], err_s_label: 1.362052, err_s_domain: 1.285986, err_t_domain: 1.215652
epoch: 0, [iter: 8 / all 95], err_s_label: 1.287921, err_s_domain: 1.227676, err_t_domain: 1.164692
epoch: 0, [iter: 9 / all 95], err_s_label: 1.205485, err_s_domain: 1.179570, err_t_domain: 1.133780
epoch: 0, [iter: 10 / all 95], err_s_label: 1.201994, err_s_domain: 1.171341, err_t_domain: 1.110068

 10%|█         | 1/10 [02:16<20:30, 136.69s/it]

epoch: 0, accuracy of the train dataset: 0.696800
epoch: 0, accuracy of the test dataset: 0.365233
epoch: 1, [iter: 1 / all 95], err_s_label: 0.641659, err_s_domain: 0.692163, err_t_domain: 0.717978
epoch: 1, [iter: 2 / all 95], err_s_label: 0.634265, err_s_domain: 0.680781, err_t_domain: 0.711848
epoch: 1, [iter: 3 / all 95], err_s_label: 0.557838, err_s_domain: 0.690271, err_t_domain: 0.717988
epoch: 1, [iter: 4 / all 95], err_s_label: 0.714094, err_s_domain: 0.688289, err_t_domain: 0.735143
epoch: 1, [iter: 5 / all 95], err_s_label: 0.512896, err_s_domain: 0.707680, err_t_domain: 0.706661
epoch: 1, [iter: 6 / all 95], err_s_label: 0.726242, err_s_domain: 0.682948, err_t_domain: 0.703635
epoch: 1, [iter: 7 / all 95], err_s_label: 0.629117, err_s_domain: 0.688001, err_t_domain: 0.719928
epoch: 1, [iter: 8 / all 95], err_s_label: 0.515379, err_s_domain: 0.681024, err_t_domain: 0.709272
epoch: 1, [iter: 9 / all 95], err_s_label: 0.533702, err_s_domain: 0.687570, err_t_domain: 0.705042
e

 20%|██        | 2/10 [04:33<18:15, 136.93s/it]

...

epoch: 7, accuracy of the train dataset: 0.881227
epoch: 7, accuracy of the test dataset: 0.651565
epoch: 8, [iter: 1 / all 95], err_s_label: 0.044903, err_s_domain: 0.689562, err_t_domain: 0.705481
epoch: 8, [iter: 2 / all 95], err_s_label: 0.072118, err_s_domain: 0.695457, err_t_domain: 0.696136
epoch: 8, [iter: 3 / all 95], err_s_label: 0.065833, err_s_domain: 0.689529, err_t_domain: 0.700179
epoch: 8, [iter: 4 / all 95], err_s_label: 0.028066, err_s_domain: 0.696683, err_t_domain: 0.691928
epoch: 8, [iter: 5 / all 95], err_s_label: 0.093987, err_s_domain: 0.692628, err_t_domain: 0.696997
epoch: 8, [iter: 6 / all 95], err_s_label: 0.200837, err_s_domain: 0.700679, err_t_domain: 0.692529
epoch: 8, [iter: 7 / all 95], err_s_label: 0.308518, err_s_domain: 0.711628, err_t_domain: 0.693758
epoch: 8, [iter: 8 / all 95], err_s_label: 0.203024, err_s_domain: 0.692275, err_t_domain: 0.692402
epoch: 8, [iter: 9 / all 95], err_s_label: 0.054836, err_s_domain: 0.697259, err_t_domain: 0.699690
e

 90%|█████████ | 9/10 [20:18<02:15, 135.09s/it]

epoch: 8, accuracy of the train dataset: 0.907126
epoch: 8, accuracy of the test dataset: 0.664800
epoch: 9, [iter: 1 / all 95], err_s_label: 0.018781, err_s_domain: 0.678257, err_t_domain: 0.702219
epoch: 9, [iter: 2 / all 95], err_s_label: 0.064548, err_s_domain: 0.687505, err_t_domain: 0.704935
epoch: 9, [iter: 3 / all 95], err_s_label: 0.097663, err_s_domain: 0.678908, err_t_domain: 0.700480
epoch: 9, [iter: 4 / all 95], err_s_label: 0.049656, err_s_domain: 0.682236, err_t_domain: 0.701988
epoch: 9, [iter: 5 / all 95], err_s_label: 0.123367, err_s_domain: 0.686777, err_t_domain: 0.709087
epoch: 9, [iter: 6 / all 95], err_s_label: 0.036280, err_s_domain: 0.686549, err_t_domain: 0.699450
epoch: 9, [iter: 7 / all 95], err_s_label: 0.053827, err_s_domain: 0.687676, err_t_domain: 0.706604
epoch: 9, [iter: 8 / all 95], err_s_label: 0.197024, err_s_domain: 0.687776, err_t_domain: 0.708497
epoch: 9, [iter: 9 / all 95], err_s_label: 0.089716, err_s_domain: 0.697974, err_t_domain: 0.697710
e

100%|██████████| 10/10 [22:32<00:00, 135.26s/it]

epoch: 9, accuracy of the train dataset: 0.945398
epoch: 9, accuracy of the test dataset: 0.698651
done
